In [4]:
"""
Full pipeline:
- Load new_merged_from_time_series.csv
- Clean & impute (no deletion): interpolation per-country -> forward/backfill -> region median -> global median
- Add imputation flags (per variable)
- Feature engineering (HDI_sq, HDI_centered, log GDP, lags, pct_change, 3-yr rolling)
- EDA: summary, correlations, plots saved to ./final_output/
- Modelling: pooled OLS, robust OLS, panel FE (linearmodels PanelOLS if installed)
- Save final_dataset.csv and model summaries
"""

import os
from pathlib import Path
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.impute import SimpleImputer

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", font_scale=1.0)

# ---------- CONFIG ----------
INPUT_CSV = "./final_unclean.csv"   # change if path differs
OUTDIR = Path("./final_output")
OUTDIR.mkdir(parents=True, exist_ok=True)

FINAL_CSV = OUTDIR / "final_dataset.csv"
EDA_SUMMARY = OUTDIR / "eda_summary.txt"
PLOTS_DIR = OUTDIR / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

# ---------- LOAD ----------
df = pd.read_csv(INPUT_CSV)
print("Loaded:", INPUT_CSV, "shape:", df.shape)

# Standardize columns we will work with
# Ensure column names trimmed
df.columns = [c.strip() for c in df.columns]

# Show basic info
print(df.columns.tolist())

# ---------- PREPARATION ----------
# Ensure Year numeric
if 'Year' in df.columns:
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
else:
    raise ValueError("The input file must contain 'Year' column for panel structure.")

# Ensure ISO3 present (we expect ISO3)
if 'ISO3' not in df.columns:
    # optionally create from 'Country Name' via country_converter if needed
    df['ISO3'] = np.nan

# Convert common numeric columns
num_cols = []
for c in ['HDI', 'Suicide_rate', 'GDP_per_capita', 'log_GDP_per_capita']:
    if c in df.columns:
        num_cols.append(c)
        df[c] = pd.to_numeric(df[c], errors='coerce')

print("Numeric columns recognized:", num_cols)

# ---------- IMPUTATION STRATEGY ----------
# We'll impute numeric variables as follows (per variable):
# 1) Per-country linear interpolation across Year (sorted) for small gaps
# 2) Per-country forward-fill then back-fill (to carry last known value)
# 3) If still missing, fill with region (continent) median for that Year (if available)
# 4) If still missing, fill with global median (or mean) — but we record imputation
# For categorical (income_group_auto, Low_data_quality_flag, continent): fill missing with mode per region or 'Unknown'

# We'll create imputation flag columns: <var>_imputed (bool) and <var>_imputed_method (text)

df_out = df.copy()

# Helper: create per-var flags and apply steps
def impute_numeric_panel(df_panel, var, country_col='ISO3', year_col='Year', region_col='continent'):
    """
    Returns df with var imputed. Adds columns:
    var + "_imputed" (bool)
    var + "_imputed_method" (str)
    """
    dfp = df_panel.copy()
    flag_col = f"{var}_imputed"
    method_col = f"{var}_imputed_method"
    dfp[flag_col] = False
    dfp[method_col] = "original"

    # Track where originally missing
    orig_na = dfp[var].isna()

    # --- Step 1: Per-country interpolation (fixed alignment) ---
    dfp = dfp.sort_values([country_col, year_col])
    interp = (
        dfp.groupby(country_col, group_keys=True)[[year_col, var]]
        .apply(lambda g: g.set_index(year_col)[var].interpolate(method='linear', limit_direction='both'))
        .reset_index()  # reset within each group
        .rename(columns={var: f"{var}_interp"})
    )
    # Now interp contains: [ISO3, Year, var_interp]
    dfp = dfp.merge(interp, on=[country_col, year_col], how='left')
    dfp[var] = dfp[f"{var}_interp"].combine_first(dfp[var])
    dfp.drop(columns=[f"{var}_interp"], inplace=True)

    interp_mask = orig_na & dfp[var].notna()
    dfp.loc[interp_mask, flag_col] = True
    dfp.loc[interp_mask, method_col] = "interpolated"

    # --- Step 2: Forward/back fill per-country ---
    before_na = dfp[var].isna()
    dfp[var] = (
        dfp.groupby(country_col, group_keys=False)[var]
        .apply(lambda s: s.ffill().bfill())
    )
    ffill_mask = before_na & dfp[var].notna()
    dfp.loc[ffill_mask, flag_col] = True
    dfp.loc[ffill_mask, method_col] = "ffill_bfill"

    # --- Step 3: Region-year median ---
    if region_col in dfp.columns:
        region_median = (
            dfp.groupby([region_col, year_col])[var]
            .transform('median')
        )
        before_na = dfp[var].isna()
        dfp[var] = dfp[var].fillna(region_median)
        region_mask = before_na & dfp[var].notna()
        dfp.loc[region_mask, flag_col] = True
        dfp.loc[region_mask, method_col] = "region_year_median"

    # --- Step 4: Global median fallback ---
    before_na = dfp[var].isna()
    global_median = dfp[var].median(skipna=True)
    dfp[var] = dfp[var].fillna(global_median)
    global_mask = before_na & dfp[var].notna()
    dfp.loc[global_mask, flag_col] = True
    dfp.loc[global_mask, method_col] = "global_median"

    # Mark remaining missing (if any)
    still_na = dfp[var].isna()
    if still_na.any():
        dfp.loc[still_na, flag_col] = True
        dfp.loc[still_na, method_col] = "unfilled"

    return dfp[[country_col, year_col, var, flag_col, method_col]]



# Apply for each numeric var we care about
vars_to_impute = [v for v in ['HDI', 'Suicide_rate', 'GDP_per_capita', 'log_GDP_per_capita'] if v in df_out.columns]

# We'll iteratively merge results of imputation back into df_out
for var in vars_to_impute:
    print("Imputing:", var)
    res = impute_numeric_panel(df_out, var, country_col='ISO3', year_col='Year', region_col='continent')
    df_out = df_out.drop(columns=[var], errors='ignore').merge(
        res, on=['ISO3', 'Year'], how='left'
    )

# ---------- CATEGORICAL / GROUP FILL ----------
# For income_group_auto, continent, Low_data_quality_flag: fill missing with country-mode, then region-mode, then 'Unknown'
def fill_categorical(dfp, col, country_col='ISO3', region_col='continent'):
    new_flag = f"{col}_filled"
    dfp[new_flag] = False
    # try country-level mode
    country_mode = dfp.groupby(country_col)[col].agg(lambda s: s.dropna().mode().iloc[0] if s.dropna().shape[0]>0 else np.nan)
    dfp[col] = dfp.apply(lambda r: country_mode.get(r[country_col]) if pd.isna(r.get(col)) and r[country_col] in country_mode.index else r.get(col), axis=1)
    filled_country = dfp[col].notna()
    dfp.loc[filled_country, new_flag] = True

    # region-level
    if region_col in dfp.columns:
        region_mode = dfp.groupby(region_col)[col].agg(lambda s: s.dropna().mode().iloc[0] if s.dropna().shape[0]>0 else np.nan)
        dfp[col] = dfp.apply(lambda r: region_mode.get(r[region_col]) if pd.isna(r.get(col)) and r.get(region_col) in region_mode.index else r.get(col), axis=1)
        filled_region = dfp[col].notna()
        dfp.loc[filled_region, new_flag] = True

    # global fallback
    global_mode = dfp[col].mode().iloc[0] if dfp[col].dropna().shape[0]>0 else "Unknown"
    before = dfp[col].isna()
    dfp[col] = dfp[col].fillna(global_mode)
    after = dfp[col].notna()
    filled_global = before & after
    dfp.loc[filled_global, new_flag] = True

    return dfp

for cat in ['income_group_auto','continent','Low_data_quality_flag']:
    if cat in df_out.columns:
        df_out = fill_categorical(df_out, cat, country_col='ISO3', region_col='continent')

# ---------- CREATE FINAL IMPUTATION SUMMARY COLUMNS ----------
# Count for each row how many numeric vars were imputed
imputed_flag_cols = [c for c in df_out.columns if c.endswith('_imputed')]
df_out['num_imputed_numeric'] = df_out[imputed_flag_cols].sum(axis=1)

# Also store methods in compact form
method_cols = [c for c in df_out.columns if c.endswith('_imputed_method')]
df_out['imputation_methods'] = df_out[method_cols].apply(lambda row: ";".join(sorted(set([str(x) for x in row if pd.notna(x) and x!='original']))), axis=1)

# ---------- FEATURE ENGINEERING ----------
# HDI squared (if not present)
if 'HDI_sq' not in df_out.columns and 'HDI' in df_out.columns:
    df_out['HDI_sq'] = df_out['HDI']**2

# HDI centered (to reduce collinearity with HDI_sq)
if 'HDI' in df_out.columns:
    df_out['HDI_centered'] = df_out['HDI'] - df_out['HDI'].mean()

# log GDP: ensure presence and create if missing
if 'log_GDP_per_capita' not in df_out.columns and 'GDP_per_capita' in df_out.columns:
    df_out['log_GDP_per_capita'] = np.log(df_out['GDP_per_capita'].where(df_out['GDP_per_capita']>0, np.nan))
    # mark as imputed by earlier pipeline

# LAG features: lag 1 of HDI and Suicide_rate per country
df_out = df_out.sort_values(['ISO3','Year'])
for v in ['HDI','Suicide_rate','GDP_per_capita','log_GDP_per_capita']:
    if v in df_out.columns:
        lag_col = f"{v}_lag1"
        df_out[lag_col] = df_out.groupby('ISO3')[v].shift(1)
        # create indicator if lag was imputed (based on existence of original var imputation)
        imp_col = f"{v}_lag1_imputed"
        df_out[imp_col] = df_out[lag_col].isna()

# Percent change (year over year) for HDI and Suicide_rate
for v in ['HDI','Suicide_rate','GDP_per_capita']:
    if v in df_out.columns:
        df_out[f"{v}_pct_change"] = df_out.groupby('ISO3')[v].pct_change()

# 3-year rolling mean (centered) where available
for v in ['HDI','Suicide_rate']:
    if v in df_out.columns:
        df_out[f"{v}_roll3"] = df_out.groupby('ISO3')[v].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)

# Z-score within-year (for comparability across countries)
for v in ['HDI','Suicide_rate','GDP_per_capita','log_GDP_per_capita']:
    if v in df_out.columns:
        df_out[f"{v}_z_year"] = df_out.groupby('Year')[v].transform(lambda x: (x - x.mean())/x.std(ddof=0))

# ---------- ETHICAL / TRANSPARENCY COLUMNS ----------
# Add column that summarizes data quality: number of imputation methods + Low_data_quality_flag_panel if exists
if 'num_imputed_numeric' not in df_out.columns:
    df_out['num_imputed_numeric'] = np.nan
df_out['data_quality_summary'] = df_out.apply(
    lambda r: f"{r.get('Low_data_quality_flag_panel', r.get('Low_data_quality_flag','Unknown'))} | imputed_vars={int(r['num_imputed_numeric'])} | methods={r.get('imputation_methods','')}",
    axis=1
)

# ---------- EDA: summary & plots ----------
# Save simple EDA summary text
with open(EDA_SUMMARY, "w", encoding="utf-8") as f:
    f.write("EDA Summary - final_dataset\n")
    f.write(f"Input file: {INPUT_CSV}\n")
    f.write(f"Rows: {len(df_out)}, Unique countries (ISO3): {df_out['ISO3'].nunique()}\n\n")
    f.write("Numeric descriptive stats (selected vars):\n")
    desc_cols = [c for c in ['HDI','Suicide_rate','GDP_per_capita','log_GDP_per_capita'] if c in df_out.columns]
    f.write(df_out[desc_cols].describe().to_string())
    f.write("\n\nImputation overview (per variable):\n")
    for var in vars_to_impute:
        flag = f"{var}_imputed"
        if flag in df_out.columns:
            n_imputed = df_out[flag].sum()
            f.write(f"{var}: imputed count = {int(n_imputed)} / {len(df_out)}\n")
    f.write("\nNotes:\n- Imputation methods are encoded in *_imputed_method columns.\n- data_quality_summary column provides quick quality flags.\n")
print("EDA summary saved to:", EDA_SUMMARY)

# Basic plots: distribution & trend plots
# 1) HDI distribution
if 'HDI' in df_out.columns:
    plt.figure(figsize=(8,4))
    sns.histplot(df_out['HDI'].dropna(), bins=30, kde=True)
    plt.title("HDI distribution (all years)")
    plt.savefig(PLOTS_DIR / "hdi_distribution.png", dpi=150)
    plt.close()

# 2) Suicide_rate distribution
if 'Suicide_rate' in df_out.columns:
    plt.figure(figsize=(8,4))
    sns.histplot(df_out['Suicide_rate'].dropna(), bins=30, kde=True)
    plt.title("Suicide rate distribution (all years)")
    plt.savefig(PLOTS_DIR / "suicide_distribution.png", dpi=150)
    plt.close()

# 3) HDI trend sample by continent
if 'Year' in df_out.columns and 'continent' in df_out.columns and 'HDI' in df_out.columns:
    plt.figure(figsize=(10,6))
    sns.lineplot(data=df_out, x='Year', y='HDI', hue='continent', estimator='median')
    plt.title("Median HDI trend by continent")
    plt.savefig(PLOTS_DIR / "hdi_trend_by_continent.png", dpi=150)
    plt.close()

# 4) Suicide trend by continent
if 'Year' in df_out.columns and 'continent' in df_out.columns and 'Suicide_rate' in df_out.columns:
    plt.figure(figsize=(10,6))
    sns.lineplot(data=df_out, x='Year', y='Suicide_rate', hue='continent', estimator='median')
    plt.title("Median Suicide rate trend by continent")
    plt.savefig(PLOTS_DIR / "suicide_trend_by_continent.png", dpi=150)
    plt.close()

# 5) Scatter HDI vs Suicide colored by Year (sample)
if 'HDI' in df_out.columns and 'Suicide_rate' in df_out.columns:
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=df_out, x='HDI', y='Suicide_rate', hue='Year', palette='viridis', alpha=0.8)
    plt.title("HDI vs Suicide (panel)")
    plt.savefig(PLOTS_DIR / "scatter_hdi_suicide_panel.png", dpi=150)
    plt.close()

print("Saved plots to:", PLOTS_DIR)

# ---------- MODELLING ----------
# We'll run:
# 1) Pooled OLS: Suicide_rate ~ HDI + HDI_centered^2 + log_GDP_per_capita + Year dummies + continent dummies
# 2) Country Fixed Effects using linearmodels.PanelOLS if available (preferred)
# 3) Robust OLS with HC1 (statsmodels)

# Prepare modeling dataframe: keep rows with Suicide_rate & HDI present after imputation
mod_df = df_out.copy()
# Ensure numeric cast
for v in ['Suicide_rate','HDI','HDI_sq','HDI_centered','log_GDP_per_capita']:
    if v in mod_df.columns:
        mod_df[v] = pd.to_numeric(mod_df[v], errors='coerce')

# Fill small missing log_GDP with global median if still missing (we already did but ensure)
if 'log_GDP_per_capita' in mod_df.columns:
    mod_df['log_GDP_per_capita'] = mod_df['log_GDP_per_capita'].fillna(mod_df['log_GDP_per_capita'].median())

# Model formula
# Use centered HDI and its square to reduce collinearity
if 'HDI_centered' in mod_df.columns:
    mod_df['HDI_centered_sq'] = mod_df['HDI_centered']**2
    predictors = ['HDI_centered', 'HDI_centered_sq']
else:
    predictors = ['HDI', 'HDI_sq']

if 'log_GDP_per_capita' in mod_df.columns:
    predictors.append('log_GDP_per_capita')

# Year dummies and continent dummies
if 'Year' in mod_df.columns:
    year_dummies = pd.get_dummies(mod_df['Year'].astype(int).astype(str), prefix='Y', drop_first=True)
    mod_df = pd.concat([mod_df, year_dummies], axis=1)
    predictors += list(year_dummies.columns)

if 'continent' in mod_df.columns:
    cont_dummies = pd.get_dummies(mod_df['continent'], prefix='cont', drop_first=True)
    mod_df = pd.concat([mod_df, cont_dummies], axis=1)
    predictors += list(cont_dummies.columns)

# Keep only rows where dependent & main regressors exist
reg_needed = ['Suicide_rate'] + [p for p in predictors if p in mod_df.columns]
mod_df_reg = mod_df.dropna(subset=reg_needed).copy()
print("Modeling observations:", len(mod_df_reg))

# Pooled OLS with robust SE
formula = "Suicide_rate ~ " + " + ".join([p for p in predictors if p in mod_df_reg.columns])
print("Pooled OLS formula:", formula)
try:
    pooled_res = smf.ols(formula=formula, data=mod_df_reg).fit(cov_type='HC1')
    with open(OUTDIR / "model_pooled_summary.txt", "w", encoding="utf-8") as f:
        f.write(pooled_res.summary().as_text())
    print("Saved pooled OLS summary.")
except Exception as e:
    print("Pooled model failed:", e)
    pooled_res = None

# Try Panel Fixed Effects with linearmodels.PanelOLS if installed
try:
    from linearmodels.panel import PanelOLS
    # prepare multiindex
    panel_df = mod_df.set_index(['ISO3','Year'])
    # dependent and exog
    exog = panel_df[[p for p in predictors if p in panel_df.columns]]
    exog = sm.add_constant(exog)
    dep = panel_df['Suicide_rate']
    # PanelOLS with entity effects and time effects (if many years)
    panel_model = PanelOLS(dep, exog, entity_effects=True, time_effects=True)
    panel_res = panel_model.fit(cov_type='robust')
    with open(OUTDIR / "panel_fixed_effects_summary.txt", "w", encoding="utf-8") as f:
        f.write(panel_res.summary.as_text())
    print("Saved panel fixed effects summary.")
except Exception as e:
    panel_res = None
    print("PanelOLS not available or failed:", e)
    # Fallback: OLS with entity dummies (country dummies)
    try:
        # create country dummies (but beware many dummies)
        country_dummies = pd.get_dummies(mod_df_reg.index.get_level_values('ISO3') if hasattr(mod_df_reg.index, 'get_level_values') else mod_df_reg['ISO3'], prefix='c', drop_first=True)
    except Exception:
        country_dummies = pd.get_dummies(mod_df_reg['ISO3'], prefix='c', drop_first=True)
    try:
        mod_with_c = pd.concat([mod_df_reg.reset_index(drop=True), country_dummies.reset_index(drop=True)], axis=1)
        preds2 = [p for p in predictors if p in mod_with_c.columns] + list(country_dummies.columns)
        formula2 = "Suicide_rate ~ " + " + ".join(preds2)
        res_with_country = smf.ols(formula=formula2, data=mod_with_c).fit(cov_type='HC1')
        with open(OUTDIR / "model_with_country_dummies_summary.txt", "w", encoding="utf-8") as f:
            f.write(res_with_country.summary().as_text())
        print("Saved OLS with country dummies summary (fallback).")
    except Exception as e2:
        print("Fallback country-dummy model failed:", e2)

# ---------- DIAGNOSTICS ----------
# VIF on predictors (excluding dummies if many)
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_covars = [p for p in predictors if p in mod_df_reg.columns and not p.startswith('Y_') and not p.startswith('cont_')]
if vif_covars:
    X_vif = sm.add_constant(mod_df_reg[vif_covars].fillna(0))
    vif_df = pd.DataFrame({
        'variable': X_vif.columns,
        'VIF': [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    })
    vif_df.to_csv(OUTDIR / "vif.csv", index=False)

# Heteroskedasticity: Breusch-Pagan
if pooled_res is not None:
    try:
        bp = sm.stats.diagnostic.het_breuschpagan(pooled_res.resid, pooled_res.model.exog)
        with open(OUTDIR / "breusch_pagan.txt", "w", encoding="utf-8") as f:
            f.write(f"LM stat: {bp[0]}, p-value: {bp[1]}, fvalue: {bp[2]}, f p-value: {bp[3]}")
    except Exception as e:
        print("BP test failed:", e)

# ---------- SAVE final dataset ----------
# Keep a well-documented final CSV with imputation flags and new features
save_cols = df_out.columns.tolist()
df_out.to_csv(FINAL_CSV, index=False)
print("Saved final dataset to:", FINAL_CSV)

# Brief final printout
print("Pipeline complete. Outputs in:", OUTDIR)
print("Final dataset shape:", df_out.shape)


Loaded: ./final_unclean.csv shape: (1175, 14)
['Country Name', 'ISO3', 'Year', 'HDI', 'Suicide_rate', 'GDP_per_capita', 'log_GDP_per_capita', 'income_group_auto', 'continent', 'Low_data_quality_flag', 'HDI_sq', 'Suicide_rate_lag1', 'HDI_lag1', 'Low_data_quality_flag_panel']
Numeric columns recognized: ['HDI', 'Suicide_rate', 'GDP_per_capita', 'log_GDP_per_capita']
Imputing: HDI
Imputing: Suicide_rate
Imputing: GDP_per_capita
Imputing: log_GDP_per_capita
EDA summary saved to: final_output\eda_summary.txt
Saved plots to: final_output\plots
Modeling observations: 1175
Pooled OLS formula: Suicide_rate ~ HDI_centered + HDI_centered_sq + log_GDP_per_capita + Y_2019 + Y_2020 + Y_2021 + Y_2022 + Y_2023 + cont_America + cont_Asia + cont_Europe + cont_Oceania
Saved pooled OLS summary.
PanelOLS not available or failed: No module named 'linearmodels'
Fallback country-dummy model failed: invalid syntax (<unknown>, line 1)
Saved final dataset to: final_output\final_dataset.csv
Pipeline complete. Out